## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [94]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



''

In [95]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A3B4B44D60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A3B4B44CA0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [96]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Aman and I am a Student of B.Tech in Computer Science.")])

AIMessage(content="Hello Aman,\n\nIt's nice to meet you!\n\nBeing a B.Tech student in Computer Science is exciting. What are you most interested in learning about within computer science?  \n\nPerhaps you're interested in:\n\n* **Artificial Intelligence?**\n* **Data Science?**\n* **Cybersecurity?**\n* **Software Development?**\n\nI'd love to hear more about your goals and aspirations.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 28, 'total_tokens': 119, 'completion_time': 0.165454545, 'prompt_time': 0.00508562, 'queue_time': 0.305072311, 'total_time': 0.170540165}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4b1b321b-0ad1-48ae-b3e7-0c3d1dff7dc9-0', usage_metadata={'input_tokens': 28, 'output_tokens': 91, 'total_tokens': 119})

In [97]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Aman and I am a Student of B.Tech in Computer Science."),
        AIMessage(content="Hi Aman,\n\nIt's nice to meet you!  That's great, B.Tech in Computer Science is a fascinating field.\n\nWhat are you most interested in learning about within computer science?  \n\nMaybe we can chat about some cool projects, technologies, or even just general advice for a student like yourself.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Aman, and you are a B.Tech student studying Computer Science!  \n\nIs there anything else you'd like to tell me about yourself? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 117, 'total_tokens': 157, 'completion_time': 0.072727273, 'prompt_time': 0.00572817, 'queue_time': 0.24803807599999997, 'total_time': 0.078455443}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9ff19305-c2f4-4484-95e5-fea070cc5994-0', usage_metadata={'input_tokens': 117, 'output_tokens': 40, 'total_tokens': 157})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [98]:
!pip install langchain_community

In [127]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [128]:
config={"configurable":{"session_id":"chat1"}}

In [129]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Aman and I am a Student of B.Tech in Computer Science.")],
    config=config
)

In [130]:
response.content

"Hi Aman,\n\nIt's nice to meet you!  \n\nStudying Computer Science is fascinating.  What are you most enjoying about your B.Tech program so far? Are there any particular areas of computer science that you're particularly interested in?\n"

In [132]:
with_message_history.invoke(
    [HumanMessage(content="What's my transformers?")],
    config=config,
)

AIMessage(content='That\'s a great question!  \n\n"Transformers" in the world of computer science refers to a type of **artificial neural network architecture** that has revolutionized natural language processing (NLP). \n\nHere\'s a simplified explanation:\n\n* **What they do:** Transformers are designed to understand and generate human language. They can be used for tasks like:\n    * **Machine translation:** Translating text from one language to another\n    * **Text summarization:**  Condensing large amounts of text into shorter summaries\n    * **Chatbots:**  Creating conversational agents that can understand and respond to human input\n    * **Text generation:** Writing stories, articles, or even code\n\n* **Key features:** \n    * **Attention mechanism:** This is what makes transformers special. It allows them to focus on the most important parts of a sentence when processing information. Think of it like humans paying attention to certain words in a conversation to understand t

In [135]:
tell=with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
tell.content

"Your name is Aman.  \n\nI remember! 😊  \n\nIs there anything else you'd like to talk about?  Perhaps you have a question about transformers or something else entirely?  I'm here to help! \n"

In [104]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to remember it for our current interaction. 😊  \n\nWhat's your name?\n"

In [105]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋\n\nIs there anything I can help you with today?\n"

In [106]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John, remember?  😄 \n\nI'm happy to help you with anything else you need. \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [107]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [108]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Aman")]})

AIMessage(content="Hello Aman! 👋 \n\nIt's nice to meet you.\n\nI'm happy to help with any questions you have. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 31, 'total_tokens': 67, 'completion_time': 0.065454545, 'prompt_time': 0.002364726, 'queue_time': 0.248881624, 'total_time': 0.067819271}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f1b89c55-a6bb-4a58-a2b9-74a7c2feeda0-0', usage_metadata={'input_tokens': 31, 'output_tokens': 36, 'total_tokens': 67})

In [109]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [110]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Aman")],
    config=config
)

response

AIMessage(content="Hello Aman! It's nice to meet you. 😊\n\nI'm here to help with any questions you have. Just ask away! \n\nWhat can I do for you today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.002348336, 'queue_time': 0.243650614, 'total_time': 0.082348336}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c69e3760-0c67-4aa6-90eb-ec8b18199af9-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [111]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Aman.  I remember!  😄  \n\nIs there anything else I can help you with?\n'

In [112]:
## Add more complexity

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [113]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Aman")],"language":"Hindi"})
response.content

'नमस्ते अमन! \n\nमुझे बताया गया है कि मैं आपकी मदद करने के लिए यहाँ हूँ। आप कोई भी सवाल पूछ सकते हैं, और मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे अच्छा उत्तर दूँ। \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [114]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [115]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Aman")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते अमन! \n\nमुझे खुशी है कि आप मुझसे जुड़े हैं। आप क्या जानना चाहते हैं? 😊 \n'

In [116]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [117]:
response.content

'आपका नाम अमन है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [118]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [119]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor. \n\nWhat's your favorite ice cream? 😊\n"

In [120]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".  \n\n\n\nI\'m ready for another one if you are! 😊\n'

In [121]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [122]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [123]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help!  😊  What can I solve for you?  \n\n"